In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
from IPython.display import display
import time
import os
from pandas import ExcelWriter

#爬外幣匯率
url = "https://rate.bot.com.tw/xrt?Lang=zh-TW"

resp = requests.get(url)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
rate_table = html.find('table', attrs={'title':'牌告匯率'}).find('tbody').find_all('tr')

buy_list = []
sell_list= []
currency_list = []

for rate in rate_table:
    prices = rate.find_all("td")
    buy_list.append(prices[1].text)
    sell_list.append(prices[2].text)

    currency_name = rate.find_all(class_="visible-phone print_hide")
    currency_list.append(currency_name[1].text.strip())

df = pd.DataFrame()
df["貨幣"] = currency_list
df["即期買入"] = buy_list
df["即期賣出"] = sell_list
# display(df)

In [ ]:
def oversea_USA_EUROPE():

    #建構爬取網站及參數
    url = 'https://www.capitalfutures.com.tw/product/deposit.asp?xy=2&xt=2'
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'
        }

    res = requests.get(url=url, headers=header)
    res.encoding = res.apparent_encoding

    # print(res.text)
    
    #建立USA_EUROPE表格
    USA_EUROPE = pd.DataFrame(columns=['代號','商品種類','幣別','原始保證金','維持保證金','當沖保證金','原始保證金(台幣)'], dtype='str')
    en_code = []
    tw_name = []
    currency = []
    initial = []
    maintainance = []
    daytrade = []
    tw_money = []

    #用正則表達式爬取

    patt = r'<tr bgcolor=".*?">.*?<td align="center">(.*?)</td>.*?<td align="center">(.*?)</td>.*?<td align=center>(.*?)</td>.*?<td align=right>(.*?)</td>.*?<td align=right>(.*?)</td>.*?<td align=right>(.*?)</td>.*?</tr>'
    res_1 = re.findall(pattern=patt, string=res.text, flags=re.S)
    
    for futures in res_1:
        temp_en_code = futures[0].replace(" ", "")
        temp_tw_name = futures[1].replace(" ", "")
        temp_currency = futures[2].replace(" ", "")
        temp_initial = futures[3].replace(" ", "")
        temp_maintainance = futures[4].replace(" ", "")
        temp_daytrade = futures[5].replace(" ", "")

        #將保證金計算為台幣

        if temp_currency == 'USD':
            temp_tw_money = int(temp_initial) * float(df.iloc[0,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'GBP':
            temp_tw_money = int(temp_initial) * float(df.iloc[2,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'JPY':
            temp_tw_money = int(temp_initial) * float(df.iloc[7,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'EUR':
            temp_tw_money = int(temp_initial) * float(df.iloc[14,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'GBP':
            temp_tw_money = int(temp_initial) * float(df.iloc[6,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'AUD':
            temp_tw_money = int(temp_initial) * float(df.iloc[3,2])
            temp_tw_money_int = int(temp_tw_money)

        en_code.append(temp_en_code)
        tw_name.append(temp_tw_name)
        currency.append(temp_currency)
        initial.append(temp_initial)
        maintainance.append(temp_maintainance)
        daytrade.append(temp_daytrade)
        tw_money.append(str(temp_tw_money_int))

    #將list寫入TW_Index
    USA_EUROPE['代號'] = en_code
    USA_EUROPE['商品種類'] = tw_name
    USA_EUROPE['幣別'] = currency
    USA_EUROPE['原始保證金'] = initial
    USA_EUROPE['維持保證金'] = maintainance
    USA_EUROPE['當沖保證金'] = daytrade
    USA_EUROPE['原始保證金(台幣)'] = tw_money

    # display(USA_EUROPE)

    #找到含有USA_EUROPE的舊檔案
    path = r"C:\Users\WEI CHIEH\Desktop\Robert Python"
    for filepath, folders, files in os.walk(path):
        for file in files:
            if 'USA_EUROPE' in file:
                compare_file = os.path.join(filepath,file)

    #開啟之前爬蟲的檔案進行比較
    compare_db = pd.read_excel(compare_file, dtype='str', index_col=0)
    result = USA_EUROPE[(USA_EUROPE['原始保證金'] == compare_db['原始保證金']) == False] #只比較原始保證金那一列
    # display(compare_db)
    # print(USA_EUROPE['原始保證金'] == compare_db['原始保證金'])
    # display(result)

    if result.empty:
        print("******************************************"*2)
        print("USA_EUROPE的資料已是最新狀態囉！")
        print("******************************************"*2)
    else:
        mask = '%Y%m%d'
        dte = datetime.now().strftime(mask)
        fname = "{}_USA_EUROPE.xlsx".format(dte)

        with pd.ExcelWriter(fname) as writer:
            USA_EUROPE.to_excel(writer, sheet_name="USA_EUROPE")

        print("******************************************"*2)
        display(result)
        print("******************************************"*2)
        print("USA_EUROPE已下載最新資料，需要更新保證金喔！")
        print("******************************************"*2)

In [ ]:
def oversea_Singapore():

    #建構爬取網站及參數
    url = 'https://www.capitalfutures.com.tw/product/deposit_sp.asp?xy=2&xt=4'
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'
        }

    res = requests.get(url=url, headers=header)
    res.encoding = res.apparent_encoding

    # print(res.text)

    #建立USA_EUROPE表格
    db_singapore = pd.DataFrame(columns=['代號','商品種類','幣別','原始保證金','維持保證金','當沖保證金','原始保證金(台幣)'], dtype='str')
    en_code = []
    tw_name = []
    currency = []
    initial = []
    maintainance = []
    daytrade = []
    tw_money = []

    #用正則表達式爬取

    patt = r'<tr bgcolor=".*?">.*?<td align="center">(.*?)</td>.*?<td align="center">(.*?)</td>.*?<td align=center>(.*?)</td>.*?<td align=right>(.*?)</td>.*?<td align=right>(.*?)</td>.*?<td align=right>(.*?)</td>.*?</tr>'
    res_1 = re.findall(pattern=patt, string=res.text, flags=re.S)
    
    for futures in res_1:
        temp_en_code = futures[0].replace(" ", "")
        temp_tw_name = futures[1].replace(" ", "")
        temp_currency = futures[2].replace(" ", "")
        temp_initial = futures[3].replace(",", "")
        temp_maintainance = futures[4].replace(",", "")
        temp_daytrade = futures[5].replace(",", "")
        
        #將保證金計算為台幣

        if temp_currency == 'USD':
            temp_tw_money = int(temp_initial) * float(df.iloc[0,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'GBP':
            temp_tw_money = int(temp_initial) * float(df.iloc[2,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'JPY':
            temp_tw_money = int(temp_initial) * float(df.iloc[7,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'EUR':
            temp_tw_money = int(temp_initial) * float(df.iloc[14,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'GBP':
            temp_tw_money = int(temp_initial) * float(df.iloc[6,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'AUD':
            temp_tw_money = int(temp_initial) * float(df.iloc[3,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'SGD':
            temp_tw_money = int(temp_initial) * float(df.iloc[5,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'RMB':
            temp_tw_money = int(temp_initial) * float(df.iloc[18,2])
            temp_tw_money_int = int(temp_tw_money)
            
        en_code.append(temp_en_code)
        tw_name.append(temp_tw_name)
        currency.append(temp_currency)
        initial.append(temp_initial)
        maintainance.append(temp_maintainance)
        daytrade.append(temp_daytrade)
        tw_money.append(str(temp_tw_money_int))

    #將list寫入TW_Index
    db_singapore['代號'] = en_code
    db_singapore['商品種類'] = tw_name
    db_singapore['幣別'] = currency
    db_singapore['原始保證金'] = initial
    db_singapore['維持保證金'] = maintainance
    db_singapore['當沖保證金'] = daytrade
    db_singapore['原始保證金(台幣)'] = tw_money

    # display(db_singapore)
    
    #找到含有Singapore的舊檔案
    path = r"C:\Users\WEI CHIEH\Desktop\Robert Python"
    for filepath, folders, files in os.walk(path):
        for file in files:
            if '_singapore' in file:
                compare_file = os.path.join(filepath,file)

    #開啟之前爬蟲的檔案進行比較
    compare_db = pd.read_excel(compare_file, dtype='str', index_col=0)
    result = db_singapore[(db_singapore['原始保證金'] == compare_db['原始保證金']) == False] #只比較原始保證金那一列
    # display(compare_db)
    # print(USA_EUROPE['原始保證金'] == compare_db['原始保證金'])
    # display(result)

    if result.empty:
        print("******************************************"*2)
        print("Singapore的資料已是最新狀態囉！")
        print("******************************************"*2)
    else:
        mask = '%Y%m%d'
        dte = datetime.now().strftime(mask)
        fname = "{}_singapore.xlsx".format(dte)

        with pd.ExcelWriter(fname) as writer:
            db_singapore.to_excel(writer, sheet_name="singapore")

        print("******************************************"*2)
        display(result)
        print("******************************************"*2)
        print("Singapore已下載最新資料，需要更新保證金喔！")
        print("******************************************"*2)
    

In [ ]:
def oversea_HongKong():

    #建構爬取網站及參數
    url = 'https://www.capitalfutures.com.tw/product/deposit-hk.asp?xy=2&xt=5'
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'
        }

    res = requests.get(url=url, headers=header)
    res.encoding = res.apparent_encoding

    # print(res.text)

    #建立USA_EUROPE表格
    db_hongkong = pd.DataFrame(columns=['代號','商品種類','幣別','原始保證金','維持保證金','當沖保證金','原始保證金(台幣)'], dtype='str')
    en_code = []
    tw_name = []
    currency = []
    initial = []
    maintainance = []
    daytrade = []
    tw_money = []

    #用正則表達式爬取

    patt = r'<tr bgcolor=".*?">.*?<td align="center">(.*?)</td>.*?<td align="center">(.*?)</td>.*?<td align=center>(.*?)</td>.*?<td align=right>(.*?)</td>.*?<td align=right>(.*?)</td>.*?<td align=right>(.*?)</td>.*?</tr>'
    res_1 = re.findall(pattern=patt, string=res.text, flags=re.S)
    
    for futures in res_1:
        temp_en_code = futures[0].replace(" ", "")
        temp_tw_name = futures[1].replace(" ", "")
        temp_currency = futures[2].replace(" ", "")
        temp_initial = futures[3].replace(",", "")
        temp_maintainance = futures[4].replace(",", "")
        temp_daytrade = futures[5].replace(",", "")
        
        #將保證金計算為台幣

        if temp_currency == 'USD':
            temp_tw_money = int(temp_initial) * float(df.iloc[0,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'GBP':
            temp_tw_money = int(temp_initial) * float(df.iloc[2,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'JPY':
            temp_tw_money = int(temp_initial) * float(df.iloc[7,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'EUR':
            temp_tw_money = int(temp_initial) * float(df.iloc[14,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'GBP':
            temp_tw_money = int(temp_initial) * float(df.iloc[6,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'AUD':
            temp_tw_money = int(temp_initial) * float(df.iloc[3,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'SGD':
            temp_tw_money = int(temp_initial) * float(df.iloc[5,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'RMB':
            temp_tw_money = int(temp_initial) * float(df.iloc[18,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'HKD':
            temp_tw_money = int(temp_initial) * float(df.iloc[1,2])
            temp_tw_money_int = int(temp_tw_money)
            
        en_code.append(temp_en_code)
        tw_name.append(temp_tw_name)
        currency.append(temp_currency)
        initial.append(temp_initial)
        maintainance.append(temp_maintainance)
        daytrade.append(temp_daytrade)
        tw_money.append(str(temp_tw_money_int))

    #將list寫入TW_Index
    db_hongkong['代號'] = en_code
    db_hongkong['商品種類'] = tw_name
    db_hongkong['幣別'] = currency
    db_hongkong['原始保證金'] = initial
    db_hongkong['維持保證金'] = maintainance
    db_hongkong['當沖保證金'] = daytrade
    db_hongkong['原始保證金(台幣)'] = tw_money

    # display(db_hongkong)

    #找到含有_hongkong的舊檔案
    path = r"C:\Users\WEI CHIEH\Desktop\Robert Python"
    for filepath, folders, files in os.walk(path):
        for file in files:
            if '_hongkong' in file:
                compare_file = os.path.join(filepath,file)

    #開啟之前爬蟲的檔案進行比較
    compare_db = pd.read_excel(compare_file, dtype='str', index_col=0)
    result = db_hongkong[(db_hongkong['原始保證金'] == compare_db['原始保證金']) == False] #只比較原始保證金那一列
    # display(compare_db)
    # print(USA_EUROPE['原始保證金'] == compare_db['原始保證金'])
    # display(result)

    if result.empty:
        print("******************************************"*2)
        print("Hongkong的資料已是最新狀態囉！")
        print("******************************************"*2)
    else:
        mask = '%Y%m%d'
        dte = datetime.now().strftime(mask)
        fname = "{}_hongkong.xlsx".format(dte)

        with pd.ExcelWriter(fname) as writer:
            db_hongkong.to_excel(writer, sheet_name="hongkong")

        print("******************************************"*2)
        display(result)
        print("******************************************"*2)
        print("Hongkong已下載最新資料，需要更新保證金喔！")
        print("******************************************"*2)
    

In [ ]:
def oversea_Japan():

    #建構爬取網站及參數
    url = 'https://www.capitalfutures.com.tw/product/deposit-jp.asp?xy=2&xt=3'
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'
        }

    res = requests.get(url=url, headers=header)
    res.encoding = res.apparent_encoding

    # print(res.text)

    #建立TOCOM表格
    db_Tocom = pd.DataFrame(columns=['代號','商品種類','幣別','原始保證金','維持保證金','原始保證金(台幣)'], dtype='str')
    en_code = []
    tw_name = []
    currency = []
    initial = []
    maintainance = []
    tw_money = []

    #用正則表達式爬取TOCOM

    patt = r'<TR bgcolor=".*?">.*?<TD align=middle rowSpan=2 width="70">(.*?)</TD>.*?<TD align=middle rowSpan=2 width="69">(.*?)</TD>.*?<TD align=middle rowSpan=2 width="70">(.*?)</TD>.*?<TD align=middle width="122">.*?</TD>.*?<td align="right" width="81">(.*?)</td>.*?<td align="right" width="85">(.*?)</td>.*?</TR>'
    res_1 = re.findall(pattern=patt, string=res.text, flags=re.S)
    
    for futures in res_1:
        temp_en_code = futures[0].replace(" ", "")
        temp_tw_name = futures[1].replace(" ", "")
        temp_currency = futures[2].replace(" ", "")
        temp_initial = futures[3].replace(",", "")
        temp_maintainance = futures[4].replace(",", "")
        
        #將保證金計算為台幣

        if temp_currency == 'USD':
            temp_tw_money = int(temp_initial) * float(df.iloc[0,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'GBP':
            temp_tw_money = int(temp_initial) * float(df.iloc[2,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'JPY':
            temp_tw_money = int(temp_initial) * float(df.iloc[7,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'EUR':
            temp_tw_money = int(temp_initial) * float(df.iloc[14,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'GBP':
            temp_tw_money = int(temp_initial) * float(df.iloc[6,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'AUD':
            temp_tw_money = int(temp_initial) * float(df.iloc[3,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'SGD':
            temp_tw_money = int(temp_initial) * float(df.iloc[5,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'RMB':
            temp_tw_money = int(temp_initial) * float(df.iloc[18,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'HKD':
            temp_tw_money = int(temp_initial) * float(df.iloc[1,2])
            temp_tw_money_int = int(temp_tw_money)
            
        en_code.append(temp_en_code)
        tw_name.append(temp_tw_name)
        currency.append(temp_currency)
        initial.append(temp_initial)
        maintainance.append(temp_maintainance)
        tw_money.append(str(temp_tw_money_int))
    
    #將list寫入TW_Index
    db_Tocom['代號'] = en_code
    db_Tocom['商品種類'] = tw_name
    db_Tocom['幣別'] = currency
    db_Tocom['原始保證金'] = initial
    db_Tocom['維持保證金'] = maintainance
    db_Tocom['原始保證金(台幣)'] = tw_money

    # display(db_Tocom)

    #建立OSE表格
    db_OSE = pd.DataFrame(columns=['代號','商品種類','幣別','原始保證金','維持保證金','當沖保證金','原始保證金(台幣)'], dtype='str')
    en_code_2 = []
    tw_name_2 = []
    currency_2 = []
    initial_2 = []
    maintainance_2 = []
    daytrade_2 = []
    tw_money_2 = []

    #用正則表達式爬取

    patt_2 = r'<tr bgcolor=".*?">.*?<td align="center">(.*?)</td>.*?<td align="center">(.*?)</td>.*?<td align=center>(.*?)</td>.*?<td align=right>(.*?)</td>.*?<td align=right>(.*?)</td>.*?<td align=right>(.*?)</td>.*?</tr>'
    res_2 = re.findall(pattern=patt_2, string=res.text, flags=re.S)

    for futures in res_2:
        temp_en_code = futures[0].replace(" ", "")
        temp_tw_name = futures[1].replace(" ", "")
        temp_currency = futures[2].replace(" ", "")
        temp_initial = futures[3].replace(",", "")
        temp_maintainance = futures[4].replace(",", "")
        temp_daytrade = futures[5].replace(",", "")
        
        #將保證金計算為台幣

        if temp_currency == 'USD':
            temp_tw_money = int(temp_initial) * float(df.iloc[0,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'GBP':
            temp_tw_money = int(temp_initial) * float(df.iloc[2,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'JPY':
            temp_tw_money = int(temp_initial) * float(df.iloc[7,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'EUR':
            temp_tw_money = int(temp_initial) * float(df.iloc[14,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'GBP':
            temp_tw_money = int(temp_initial) * float(df.iloc[6,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'AUD':
            temp_tw_money = int(temp_initial) * float(df.iloc[3,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'SGD':
            temp_tw_money = int(temp_initial) * float(df.iloc[5,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'RMB':
            temp_tw_money = int(temp_initial) * float(df.iloc[18,2])
            temp_tw_money_int = int(temp_tw_money)
        elif temp_currency == 'HKD':
            temp_tw_money = int(temp_initial) * float(df.iloc[1,2])
            temp_tw_money_int = int(temp_tw_money)
            
        en_code_2.append(temp_en_code)
        tw_name_2.append(temp_tw_name)
        currency_2.append(temp_currency)
        initial_2.append(temp_initial)
        maintainance_2.append(temp_maintainance)
        daytrade_2.append(temp_daytrade)
        tw_money_2.append(str(temp_tw_money_int))

    #將list寫入TW_Index
    db_OSE['代號'] = en_code_2
    db_OSE['商品種類'] = tw_name_2
    db_OSE['幣別'] = currency_2
    db_OSE['原始保證金'] = initial_2
    db_OSE['維持保證金'] = maintainance_2
    db_OSE['當沖保證金'] = daytrade_2
    db_OSE['原始保證金(台幣)'] = tw_money_2

    # display(db_OSE)

    #找到含有_Japan的舊檔案
    path = r"C:\Users\WEI CHIEH\Desktop\Robert Python"
    for filepath, folders, files in os.walk(path):
        for file in files:
            if '_Japan' in file:
                compare_file = os.path.join(filepath,file)
    
    #開啟之前TOCOM爬蟲的檔案進行比較
    compare_db_TOCOM = pd.read_excel(compare_file, dtype='str', index_col=0, sheet_name='TOCOM')
    result_TOCOM = db_Tocom[(db_Tocom['原始保證金'] == compare_db_TOCOM['原始保證金']) == False] #只比較原始保證金那一列

    if result_TOCOM.empty:
        print("******************************************"*2)
        print("TOCOM的資料已是最新狀態囉！")
        print("******************************************"*2)
    else:
        mask = '%Y%m%d'
        dte = datetime.now().strftime(mask)
        fname = "{}_Japan.xlsx".format(dte)

        with pd.ExcelWriter(fname) as writer:
            db_Tocom.to_excel(writer, sheet_name="TOCOM")
            db_OSE.to_excel(writer, sheet_name="OSE")
            writer.save()

        print("******************************************"*2)
        display(result_TOCOM)
        print("******************************************"*2)
        print("TOCOM已下載最新資料，需要更新保證金喔！")
        print("******************************************"*2)

    #開啟之前OSE爬蟲的檔案進行比較
    compare_db_OSE = pd.read_excel(compare_file, dtype='str', index_col=0, sheet_name='OSE')
    result_OSE = db_OSE[(db_OSE['原始保證金'] == compare_db_OSE['原始保證金']) == False] #只比較原始保證金那一列

    if result_OSE.empty:
        print("******************************************"*2)
        print("OSE的資料已是最新狀態囉！")
        print("******************************************"*2)
    else:
        mask = '%Y%m%d'
        dte = datetime.now().strftime(mask)
        fname = "{}_Japan.xlsx".format(dte)

        with pd.ExcelWriter(fname) as writer:
            db_Tocom.to_excel(writer, sheet_name="TOCOM")
            db_OSE.to_excel(writer, sheet_name="OSE")
            writer.save()

        print("******************************************"*2)
        display(result_OSE)
        print("******************************************"*2)
        print("OSE已下載最新資料，需要更新保證金喔！")
        print("******************************************"*2)    

In [ ]:
if __name__ == '__main__' :
    
    start = time.time()
    
    oversea_USA_EUROPE()
    oversea_Singapore()
    oversea_HongKong()
    oversea_Japan()

    time_consumed = time.time() - start
    print("「總共耗時："+str(time_consumed)+" 秒」")